# Overview
The purpose of this notebook is to demonstrate how to align an LLM-As-A-Judge Prompt to human preference. It's time intensive and not scalable to use human annotators for every change. To get around this constraint, we can use an LLM to judge our own answers. We do this by using a grading rubric (similar to grading rubrics in school). You define your sucess criteria and ask the model to evaluate the results. 


#### How do we trust an LLM to grade the answers correctly? 
To use LLM-As-A-Judge, you have to iterate on the evaluation prompt until the human annotations generally agree with the LLMs grades. An evaluation dataset should be created and graded by a human. That same dataset is run through an LLM using a grading rubric. If the responses align then the evaluation prompt is ready to be used. If not, you need to iterate on the prompt until the humans and LLM agrees. 


#### What Metrics Should I Care About.
In this notebook, we'll be generating a score from 0-5 based on our grading rubric. Because of that, the metrics we care about are exact matches, scores within one point of each other, and typical metrics for numeric answers like MSE and RMSE.


# What Will We Do? 
* Curate a dataset of gold standard answers and human grades to a list of questions (we've created one already)
* Create a rubric to define how we want to grade our LLM responses
* Invoke our model using our validation dataset to generate answers
* Generate metrics to understand how well our LLM-As-A-Judge rubric matches human grades

At this end of this notebook, we should have a grading rubric that aligns to our expectations allowing us to trust this rubric moving forward

# Load Validation Dataset
This dataset contains the query, context, sample llm response, human grade, and the human thinking. We will use this to then test out our rubric to see how it compares to the human grades. 

**Note:** These initial grades were performed by Claude. However, we went through each one and updated it based our own "preference". 

In [ ]:
import pandas as pd

eval_df = pd.read_csv('../data/eval-datasets/4(a)_rubric_alignment.csv')

In [ ]:
# Lets look at the evaluation dataset
eval_df.iloc[0]

# Define Rubric

In [ ]:
# System Prompt
RUBRIC_SYSTEM_PROMPT = """You are an expert judge evaluating Retrieval Augmented Generation (RAG) applications.
Your task is to evaluate given answers based on context and questions using the criteria provided.
Evaluation Criteria (Score either 0 or 1 for each, total score is the sum):
1. Context Utilization: Does the answer use only information provided in the context, without introducing external or fabricated details?
2. Completeness: Does the answer thoroughly address all key elements of the question based on the available context, without significant omissions?
3. Conciseness: Does the answer efficiently use words to address the question and avoid unnecessary redundancy?
4. Accuracy: Is the answer factually correct based on the given context?
5. Clarity: Is the answer easy to understand and follow?
Your role is to provide a fair and thorough evaluation for each criterion, explaining your reasoning clearly."""

# User Prompt
RUBRIC_USER_PROMPT = """Please evaluate the following RAG response:

Question:
<query_text>
{query_text}
</query_text>

Generated answer:
<llm_response>
{llm_response}
</llm_response>

Context:
<context>
{context}
</context>

Evaluation Steps:
1. Carefully read the provided context, question, and answer.
2. For each evaluation criterion, assign a score of either 0 or 1:
   - Context Utilization
   - Completeness
   - Conciseness
   - Accuracy
   - Clarity
3. Provide a clear explanation for each score, referencing specific aspects of the response.
4. Calculate the total score by adding up the points awarded (minimum 0, maximum 5).
5. Present your evaluation inside <thinking></thinking> tags.
6. Include individual criterion scores (0 or 1) in the thinking tags and the total score inside <score></score> tags.
7. Ensure your response is valid XML and provides a comprehensive evaluation.

Example Output Format:
<thinking>
Context Utilization: 1 - The answer strictly uses information from the context without introducing external details.
Completeness: 1 - The response covers all key elements of the question based on the available context.
Conciseness: 1 - The answer is helpful and doesn't repeat the same information more than once.
Accuracy: 1 - All stated facts align perfectly with the provided context.
Clarity: 1 - The response is clear and easy to follow.
</thinking>
<score>4</score>

Please provide your detailed evaluation."""

## Bedrock Helpers
Below is a helper class that makes it easier for us to call bedrock in a threaded way to speed this portion up. 

In [ ]:
import boto3
import pandas as pd
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict, Any

class GradingRubricClient:
    
    def __init__(self, user_prompt: str, system_prompt: str, model_id: str, hyper_params: dict):
        self.client = boto3.client('bedrock-runtime')
        self.user_prompt = user_prompt
        self.system_prompt = system_prompt
        self.model_id = model_id
        self.hyper_params = hyper_params

    def create_chat_payload(self, inputs: dict) -> list[dict]:
        user_prompt = self.user_prompt.format(**inputs)
        user_msg = {"role": "user", "content": [{"text": user_prompt}]}
        return [user_msg]

    def call(self, messages: list[dict]) -> str:
        response = self.client.converse(
            modelId=self.model_id,
            messages=messages,
            inferenceConfig=self.hyper_params,
            system=[{
                "text": self.system_prompt
            }]
        )
        
        return response['output']['message']['content'][0]['text']

    def call_threaded(self, message_lists: List[List[Dict[str, Any]]]) -> List[str]:
        '''
        This is a bit funky. We're dumping all the requests into a thread pool
        and storing the index for the order in which they were submitted. 
        Lastly, we're inserting them into the response array at their index to ensure order.
        '''
        future_to_position = {}
        
        with ThreadPoolExecutor(max_workers=5) as executor:
            for i, request in enumerate(message_lists):
                future = executor.submit(self.call, request)
                future_to_position[future] = i
            
            responses = [None] * len(message_lists)
            for future in as_completed(future_to_position):
                position = future_to_position[future]
                try:
                    response: str = future.result()
                    responses[position] = response
                except Exception as exc:
                    print(f"Request at position {position} generated an exception: {exc}")
                    responses[position] = None
            
        return responses

    def extract_score_and_thinking(self, llm_output: str) -> tuple:
        thinking_match = re.search(r'<thinking>(.*?)</thinking>', llm_output, re.DOTALL)
        score_match = re.search(r'<score>(.*?)</score>', llm_output, re.DOTALL)

        thinking = thinking_match.group(1).strip() if thinking_match else "No thinking found"
        score = float(score_match.group(1)) if score_match else None

        return score, thinking

    def grade_examples(self, df: pd.DataFrame) -> pd.DataFrame:
        result_df = df.copy()

        message_lists = [self.create_chat_payload({
            "query_text": row["query_text"],
            "context": row["context"],
            "llm_response": row["llm_response"]
        }) for _, row in df.iterrows()]

        responses = self.call_threaded(message_lists)

        llm_scores = []
        llm_thinking = []

        for response in responses:
            if response is not None:
                score, thinking = self.extract_score_and_thinking(response)
                llm_scores.append(score)
                llm_thinking.append(thinking)
            else:
                llm_scores.append(None)
                llm_thinking.append("Error occurred during processing")

        result_df['llm_grade'] = llm_scores
        result_df['llm_reasoning'] = llm_thinking

        return result_df

# Run Validation
In this section we will run our LLM as judge prompt and compare the scores to the human annotated scores. If all goes well, our rubric should match pretty closely to our human currated scores

In [ ]:
# Define different models we can use to evaluate. 

SONNET_ID = "anthropic.claude-3-sonnet-20240229-v1:0"
HAIKU_ID = "anthropic.claude-3-haiku-20240307-v1:0"

In [ ]:
# Define our grading client
HYPER_PARAMS = {"temperature": 0.3, "maxTokens": 4096}

grading_client: GradingRubricClient = GradingRubricClient(
    RUBRIC_USER_PROMPT,
    RUBRIC_SYSTEM_PROMPT,
    HAIKU_ID,
    HYPER_PARAMS
)

experiment_1_df = grading_client.grade_examples(eval_df)

# Generate A Report
Going through these 1 by 1 is painful. We only have ~25 examples, but if you have 100+ it's not a small ask. Lets write a helper class that can generate a report for us. 

# Understanding Rubric Validation Metrics

When comparing LLM-generated grades to human-assigned grades, several metrics help us quantify the alignment and accuracy of the AI model:

1. **Mean Squared Error (MSE) & Root Mean Squared Error (RMSE)**:
   - These metrics measure the average squared difference between the LLM and human grades. They penalize larger errors more heavily, giving us insight into the magnitude of discrepancies.
   - Interpretation: Lower is better. For a 5-point scale:
     - Excellent: MSE < 0.25 (RMSE < 0.5)
     - Good: 0.25 ≤ MSE < 1 (0.5 ≤ RMSE < 1)
     - Fair: 1 ≤ MSE < 2.25 (1 ≤ RMSE < 1.5)
     - Poor: MSE ≥ 2.25 (RMSE ≥ 1.5)

2. **Mean Absolute Error (MAE)**:
   - This represents the average absolute difference between LLM and human grades, providing a straightforward measure of error magnitude.
   - Interpretation: Lower is better. For a 5-point scale:
     - Excellent: MAE < 0.5
     - Good: 0.5 ≤ MAE < 1
     - Fair: 1 ≤ MAE < 1.5
     - Poor: MAE ≥ 1.5

3. **R-squared (R²)**:
   - This metric indicates how well the LLM grades explain the variation in human grades. A higher R² suggests better alignment between the two.
   - Interpretation: Higher is better.
     - Excellent: R² > 0.9
     - Good: 0.7 < R² ≤ 0.9
     - Fair: 0.5 < R² ≤ 0.7
     - Poor: R² ≤ 0.5

4. **Pearson & Spearman Correlations**:
   - These measure the strength and direction of the relationship between LLM and human grades. High positive correlations indicate strong agreement in ranking and relative scoring.
   - Interpretation: Closer to 1 is better.
     - Excellent: > 0.9
     - Good: 0.7 - 0.9
     - Fair: 0.5 - 0.7
     - Poor: < 0.5

5. **Exact Match Ratio**:
   - This shows the proportion of cases where the LLM grade exactly matches the human grade, giving us a clear measure of perfect agreement.
   - Interpretation: Higher is better.
     - Excellent: > 0.7
     - Good: 0.5 - 0.7
     - Fair: 0.3 - 0.5
     - Poor: < 0.3

6. **Within 1 Point Ratio**:
   - This metric tells us how often the LLM grade is within one point of the human grade, allowing for slight disagreements that may not be practically significant.
   - Interpretation: Higher is better.
     - Excellent: > 0.9
     - Good: 0.8 - 0.9
     - Fair: 0.7 - 0.8
     - Poor: < 0.7

By analyzing these metrics together, we can gain a comprehensive understanding of how well the LLM rubric aligns with human grading patterns. This information is crucial for assessing the reliability and potential biases of the AI grading system, and for identifying areas where the model may need improvement or human oversight.

Note: The interpretation guidelines provided are general and may need to be adjusted based on the specific context of your grading system, the importance of precision in your application, and the inherent variability in human grading for your particular use case.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr, spearmanr

class RubricValidator:
    def __init__(self, df):
        self.df = df
        self.human_scores = self.convert_to_float(df['human_grade'])
        self.llm_scores = self.convert_to_float(df['llm_grade'])

    def convert_to_float(self, series):
        return series.astype(float)

    def calculate_metrics(self):
        mse = mean_squared_error(self.human_scores, self.llm_scores)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(self.human_scores, self.llm_scores)
        r2 = r2_score(self.human_scores, self.llm_scores)
        
        # Handle constant input for correlations. This can sometimes happen when the LLM hands out all 5.0s for example. 
        # In this case, pearsonr or spearmanr can't be computted because all the inputs are the same (i.e. constant)
        if len(set(self.human_scores)) == 1 or len(set(self.llm_scores)) == 1:
            pearson_corr = spearman_corr = "N/A (constant input)"
        else:
            pearson_corr, _ = pearsonr(self.human_scores, self.llm_scores)
            spearman_corr, _ = spearmanr(self.human_scores, self.llm_scores)
        
        exact_match = np.mean(self.human_scores == self.llm_scores)
        within_1_point = np.mean(np.abs(self.human_scores - self.llm_scores) <= 1)

        return {
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R-squared': r2,
            'Pearson Correlation': pearson_corr,
            'Spearman Correlation': spearman_corr,
            'Exact Match Ratio': exact_match,
            'Within 1 Point Ratio': within_1_point
        }

    def generate_report(self):
        metrics = self.calculate_metrics()
        report = "LLM Rubric Validation Report\n"
        report += "===========================\n\n"
        
        for metric, value in metrics.items():
            if isinstance(value, str):
                report += f"{metric}: {value}\n"
            else:
                report += f"{metric}: {value:.4f}\n"
        
        return report

    def analyze_discrepancies(self):
        discrepancies = self.df[self.human_scores != self.llm_scores]
        return discrepancies[['query_text', 'human_grade', 'llm_grade', 'human_reasoning', 'llm_reasoning']]


In [ ]:
# Run Report
validator = RubricValidator(experiment_1_df)
print(validator.generate_report())

# Experiment 1 Summary
LLMs are non-deterministic so it's possible your results vary. In our iterations, this first experiment isn't up to par. Our RMSE and pearson correlation are important, but we care about most is getting most of the answers within 1 point of each other. In our run, we got 70%. It's extremely difficult to align your rubric so that it's a 100% match. If you have more than 1 human grading results, their opinions vary and you rarely get a perfect human graded validation set. If we can get to 90%+, we'll be happy as a starting place.

We encourage you to open up the dataframe and see where the descrepancies are. 

The takeway **should** be that our grading rubric is a little too forgiving. Before we adjust the prompt though, lets see if Sonnet gives us better results.

# Experiment 2 - Use Sonnet
In this experiment, we'll use the same hyper parameters but just use sonnet instead.

In [ ]:
# Define our grading client this time with haiku
grading_client: GradingRubricClient = GradingRubricClient(
    RUBRIC_USER_PROMPT,
    RUBRIC_SYSTEM_PROMPT,
    SONNET_ID,
    HYPER_PARAMS
)

experiment_2_df = grading_client.grade_examples(eval_df)

In [ ]:
# Run Report
validator = RubricValidator(experiment_2_df)
print(validator.generate_report())

# Experiment 2 Summary
In our experiment 2 run, the results are even worse. It turns out the Sonnet really likes its own answers :). If you want, you can open up the experiment_2_df to see what's happening. Sonnet is grading every answers as a 5. Not only is this not what we want, it also makes calculating the Perason and Spearman Correlations impossible. It requires variations in the grading outputs. If it's all 5's, it's considered constant.

This is a good time to point out that LLMs in general prefer AI generated responses. We'll discuss more about how to overcome that in the conclusion

Now let's adjust our prompt and let the model know that it can be a little tougher. We can also use Haiku for this because there's no indication that Sonnet perform better. It's generally better to pick the smallest/cheapest model that gives you the performance you're looking for

# Experiment 3 - Grade more difficult
In this next step, we're going to grade the results a little harder. We'll also bump up the temperature a bit to give it more wiggle room to be creative. We'll also modify the user prompt by append a sentence to it telling the model to grade harder

In [ ]:
# Note the space is needed since the last sentence ends with .
HARDER_SUFFIX = ''' Be very tough with the grades, especially on the Conciseness and Clarity. 

If it's not very concise, give it a 0. 
If the response contains information not in the context, give it a 0.
Assume you are not a technical person when giving out the grade.

You should rarely give out a 5. There's no free handouts!'''


HARDER_USER_PROMPT = RUBRIC_USER_PROMPT + HARDER_SUFFIX

In [ ]:
# Define our grading client
HYPER_PARAMS = {"temperature": 0.6, "maxTokens": 4096}

grading_client: GradingRubricClient = GradingRubricClient(
    HARDER_USER_PROMPT,
    RUBRIC_SYSTEM_PROMPT,
    HAIKU_ID,
    HYPER_PARAMS
)

experiment_3_df = grading_client.grade_examples(eval_df)

In [ ]:
# Run Report
validator = RubricValidator(experiment_3_df)
print(validator.generate_report())

# Experiment 3 - Results
We got a much lower error, higher Pearson & spearman correlation and we got 92% of the scores within one point. For the purpose of this example, that's acceptable. It's okay if your results don't look the same. We turned the temperature up and it's not expected to get the same results every time. LLM-As-A-Judge isn't a magic bullet. For this notebook, we care more that the majority of grades are within 1 point of each other.

# Conclusion
In this notebook, we did our best to align our LLM to our own preference. It turns out, we needed our rubric to grade our results much harder. You can continue to tinker with the prompt, hyper params, and model to maximize the metrics that make sense for you.

One interesting takeaway from LLM-As-A-Judge noted above is that models tend to favor their own AI generated content. In this case, Sonnet really liked it's own answers while Haiku aligned better to our human grades. This is why it's important to test these things out with a validation dataset.

You can overcome this preference by providing human examples in your prompt. As you adjust this over time, it would be valuable to build up a repository of human currated example grades and pass them into the model as few shot examples.

# Next Steps
Move to the next notebook so we can evaluate our LLM Prompt